# Implement Kalman model using FastAI

> need to implement custom data preparation pipeline and loss function 

## Data Preparation

The aim of the data preparation pipeline is to:
- take the original time series and split it into time blocks
- for each block generate a random gap (need to figure out the properties of the gap)
- split some time blocks for testing

the input of the pipeline is:
- a dataframe containing all observations

the input of the model is:
- observed data (potentially containing NaN where data is missing)
- missing data mask (which is telling where the data is missing)
- the data needs to be standardized

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
#| default_exp kalman.fastai

In [ ]:
#| export
from meteo_imp.utils import *
from meteo_imp.gaussian import *

In [ ]:
reset_seed()

In [ ]:
import torch

In [ ]:
from fastai.tabular.core import *
from fastai.data.core import *

In [ ]:
#| export
from fastcore.transform import *
from fastcore.basics import *
from fastcore.foundation import *
from fastcore.all import *
from fastai.tabular import *
from fastai.torch_core import default_device, to_device

from meteo_imp.data import read_fluxnet_csv, hai_path

import collections

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
@cache_disk(cache_dir / "full_hai")
def load_data(dtype=np.float32):
    return read_fluxnet_csv(hai_path, None, num_dtype=dtype)

hai = load_data()
hai64 = load_data(np.float64)

### 1) Blocks

the first step is to transfrom the original dataframe into blocks of a specified `block_len`

two different strategies are possible:

- contigous blocks
- random block in the dataframe

In [ ]:
#| export
class BlockDfTransform(Transform):
    """divide timeseries DataFrame into blocks"""
    def __init__(self, df, block_len=200): 
        self.df = df 
        self.block_len = block_len
        self.n = len(df)
        
    def encodes(self, i:int) -> pd.DataFrame:       
        start = i * self.block_len
        end = (i+1) * self.block_len
        assert end <= self.n 
        
        block = self.df[start:end]
        
        return block

In [ ]:
blk = BlockDfTransform(hai, 10)

In [ ]:
blk

BlockDfTransform:
encodes: (int,object) -> encodes
decodes: 

In [ ]:
blk(1)

,TA,SW_IN,VPD
time,,,
2000-01-01 05:30:00,-0.23,0.00,0.138
2000-01-01 06:00:00,-0.23,0.00,0.122
2000-01-01 06:30:00,-0.22,0.00,0.098
2000-01-01 07:00:00,-0.24,0.00,0.066
2000-01-01 07:30:00,-0.23,0.00,0.044
2000-01-01 08:00:00,-0.22,0.00,0.026
2000-01-01 08:30:00,-0.19,0.45,0.016
2000-01-01 09:00:00,-0.14,3.70,0.010
2000-01-01 09:30:00,-0.03,7.26,0.006


In [ ]:
180 * 24 * 2 / 10

864.0

we are taking a day in the summer so there is an higher values for the variables

In [ ]:
blk(800)

,TA,SW_IN,VPD
time,,,
2000-06-15 16:30:00,14.65,468.190002,6.454
2000-06-15 17:00:00,14.22,224.800003,5.799
2000-06-15 17:30:00,14.11,195.279999,6.577
2000-06-15 18:00:00,14.23,244.169998,6.931
2000-06-15 18:30:00,14.40,253.919998,7.286
2000-06-15 19:00:00,14.09,177.309998,7.251
2000-06-15 19:30:00,13.71,97.070000,6.683
2000-06-15 20:00:00,13.08,39.709999,5.851
2000-06-15 20:30:00,12.41,10.650000,5.254


In [ ]:
tfms1 = TfmdLists([800,801,802,803], [BlockDfTransform(hai, 10)])

In [ ]:
tfms1[0]

,TA,SW_IN,VPD
time,,,
2000-06-15 16:30:00,14.65,468.190002,6.454
2000-06-15 17:00:00,14.22,224.800003,5.799
2000-06-15 17:30:00,14.11,195.279999,6.577
2000-06-15 18:00:00,14.23,244.169998,6.931
2000-06-15 18:30:00,14.40,253.919998,7.286
2000-06-15 19:00:00,14.09,177.309998,7.251
2000-06-15 19:30:00,13.71,97.070000,6.683
2000-06-15 20:00:00,13.08,39.709999,5.851
2000-06-15 20:30:00,12.41,10.650000,5.254


### 2) Gaps

adds a mask which includes a random gap

In [ ]:
class MaskedDf:
    def __init__(self,*args):
        self.data = args[0]
        self.mask = args[1]
    def __iter__(self): return iter((self.data, self.mask,))
    __repr__ = basic_repr("data, mask")
    def _repr_html_(self):
        return row_dfs({'data': self.data, 'mask': self.mask}, title="Masked Df")

In [ ]:
#| exports
def _make_random_gap(
    gap_length: int, # The length of the gap
    total_length: int, # The total number of observations
    gap_start: int = None # Optional start of gap
): # (total_length) array of bools to indicicate if the data is missing or not
    "Add a continous gap of ginve length at random position"
    if(gap_length >= total_length):
        return np.repeat(True, total_length)
    gap_start = np.random.randint(total_length - gap_length) if gap_start is None else gap_start
    return np.hstack([
        np.repeat(False, gap_start),
        np.repeat(True, gap_length),
        np.repeat(False, total_length - (gap_length + gap_start))
    ])

In [ ]:
#| export
from fastcore.basics import *

In [ ]:
#| export
class AddGapTransform(Transform):
    """Adds a random gap to a `TimeSTensor`"""
    def __init__(self,
                variables,
                gap_length,
                ):
        store_attr()
    def encodes(self, df: pd.DataFrame):
        gap = _make_random_gap(self.gap_length, df.shape[0])
        mask = np.ones_like(df, dtype=bool)
        col_sel = L(*df.columns).argwhere(lambda x: x in self.variables)
        mask[np.argwhere(gap), col_sel] = False
        return MaskedDf(df, pd.DataFrame(mask, index=df.index, columns=df.columns))

In [ ]:
a_gap = AddGapTransform(['TA', 'VPD'], 5)
a_gap

AddGapTransform:
encodes: (DataFrame,object) -> encodes
decodes: 

In [ ]:
a_gap(blk(800))

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
m_df = a_gap(blk(800))

In [ ]:
display_as_row({'data': m_df.data, 'mask': m_df.mask})

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
tfms2 = TfmdLists([800,801,802,803], [BlockDfTransform(hai, 10), AddGapTransform(['TA','SW_IN'], 2)])

In [ ]:
tfms2[0]

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
#| export
@patch
def tidy(self: MaskedDf):
    data = self.data.reset_index().melt("time")
    mask = self.mask.reset_index().melt("time", value_name="is_present")
    
    return pd.merge(data, mask, on=["time", "variable"])

In [ ]:
m_df.tidy()

,time,variable,value,is_present
0,2000-06-15 16:30:00,TA,14.650000,False
1,2000-06-15 17:00:00,TA,14.220000,False
2,2000-06-15 17:30:00,TA,14.110000,False
3,2000-06-15 18:00:00,TA,14.230000,False
4,2000-06-15 18:30:00,TA,14.400000,False
5,2000-06-15 19:00:00,TA,14.090000,True
6,2000-06-15 19:30:00,TA,13.710000,True
7,2000-06-15 20:00:00,TA,13.080000,True
8,2000-06-15 20:30:00,TA,12.410000,True
9,2000-06-15 21:00:00,TA,12.270000,True


#### Plotting

In [ ]:
#| export
import altair as alt
from altair import datum

In [ ]:
#| exporti
def def_selection():
    return alt.selection_interval(bind="scales")

In [ ]:
#| exporti
def plot_rug(df, sel = def_selection(), props = {}):
    if 'height' in props:
        props = props.copy() 
        props.pop('height') # rug should have default heigth
    return alt.Chart(df).mark_tick(
            color='black',
        ).encode(
            x = "time",
            color = alt.condition(datum.is_present, alt.value('white'), alt.value('black'))
        ).add_params(
            sel
        ).properties(**props) 

In [ ]:
plot_rug(m_df.tidy())

alt.Chart(...)

In [ ]:
def plot_points(df, y_label = "", sel = def_selection(), props = {}):
    return alt.Chart(df).mark_point(
            color='black',
            strokeWidth = 1,
            fillOpacity = 1
        ).encode(
            x = alt.X("time"),
            y = alt.Y("value", title = y_label, scale=alt.Scale(zero=False)),
            fill= alt.Fill("is_present", scale = alt.Scale(range=["black", "#ffffff00"]),
                           legend = alt.Legend(title =["Observed data"])),
            shape = "is_present",
        )

In [ ]:
plot_points(m_df.tidy())

alt.Chart(...)

In [ ]:
df = m_df.tidy()

In [ ]:
df = df[df.variable=="TA"].copy()

In [ ]:
df['row_number'] = df.reset_index().index

In [ ]:
df

,time,variable,value,is_present,row_number
0,2000-06-15 16:30:00,TA,14.65,False,0
1,2000-06-15 17:00:00,TA,14.22,False,1
2,2000-06-15 17:30:00,TA,14.11,False,2
3,2000-06-15 18:00:00,TA,14.23,False,3
4,2000-06-15 18:30:00,TA,14.40,False,4
5,2000-06-15 19:00:00,TA,14.09,True,5
6,2000-06-15 19:30:00,TA,13.71,True,6
7,2000-06-15 20:00:00,TA,13.08,True,7
8,2000-06-15 20:30:00,TA,12.41,True,8
9,2000-06-15 21:00:00,TA,12.27,True,9


In [ ]:
df[~df.is_present]

,time,variable,value,is_present,row_number
0,2000-06-15 16:30:00,TA,14.65,False,0
1,2000-06-15 17:00:00,TA,14.22,False,1
2,2000-06-15 17:30:00,TA,14.11,False,2
3,2000-06-15 18:00:00,TA,14.23,False,3
4,2000-06-15 18:30:00,TA,14.40,False,4


idea is to replace the rug with a shaded area

In [ ]:
def plot_missing_area(df, sel = def_selection(), props={}):
    pass

In [ ]:
#| exporti
def plot_line(df, only_present=True, y_label = "", sel = def_selection(), props = {}):
    # df = df[df.is_present] if only_present else df
    # TODO remove onle_present
    return alt.Chart(df).mark_line().encode(
        x = "time",    
        y = alt.Y("value", title = y_label, scale=alt.Scale(zero=False)),
        color='variable'
    ).add_params(
        sel
    ).properties(
        **props
    )#.transform_filter(
    #     datum.is_present
    # )

    

In [ ]:
plot_line(m_df.tidy())

alt.Chart(...)

In [ ]:
#| exporti
def plot_error(df, y_label = "", sel = def_selection(), props = {}):
    df.loc[:,'err_low'] = df.value - 2 * df['std']
    df.loc[:,'err_high'] = df.value + 2 * df['std']
    return alt.Chart(df).mark_errorband().encode(
        x = "time",    
        y = alt.Y("err_low:Q", title = y_label, scale=alt.Scale(zero=False)),
        y2 = "err_high:Q",
        color=alt.Color("variable",
                        legend = alt.Legend(title=["Line: pred. mean", "area: +/- 2 std"])
                       )
    ).add_params(
        sel
    ).properties(
        **props
    )
    

In [ ]:
plot_error(m_df.tidy().assign(std=5))

alt.Chart(...)

In [ ]:
#| exporti
def plot_variable(df, variable, title="", y_label="", sel = None, error=False, props = {}):
    df = df[df.variable == variable].copy()
    sel = ifnone(sel, def_selection())
    # rug = plot_rug(df, sel, props)
    points = plot_points(df, y_label, sel, props)
    line = plot_line(df, True, y_label, sel, props)
    if error: line = plot_error(df, y_label, sel, props) + line
    
    return (points + line).properties(title=title)
    
    # return alt.VConcatChart(vconcat=[(points + line), rug], spacing=-10).properties(title=title)

In [ ]:
plot_variable(m_df.tidy(), "TA", title="title TA")

alt.LayerChart(...)

In [ ]:
plot_variable(m_df.tidy().assign(std=.5), "TA", title="title TA", error=True)

alt.LayerChart(...)

In [ ]:
def facet_variable(df, # tidy dataframe
                   n_cols: int = 3,
                   bind_interaction: bool =True, # Whether the sub-plots for each variable should be connected for zooming/panning
                   props:dict = None, # additional properties for altair plot (eg. size)
                   error:bool=False # plot error bar
                   ) -> alt.Chart:
    """Plot all values of the column `variable` in different subplots"""
    props = ifnone(props, {'width': 200, 'height': 150})
    vars = df.variable.unique()
    plot_list = [alt.hconcat() for _ in range(0, len(vars), n_cols)]
    selection_scale = alt.selection_interval(bind="scales", encodings=['x']) if bind_interaction else None
    for idx, variable in enumerate(vars):
        plot = plot_variable(df,
                            variable,
                            title = variable,
                            y_label = variable,
                            sel = selection_scale,
                            props=props,
                            error=error)
        
        plot_list[idx // n_cols] |= plot
    
    plot = alt.vconcat(*plot_list)
    
    return plot

In [ ]:
#| export
@patch
def show(self: MaskedDf, ax=None, ctx=None, 
        n_cols: int = 3,
        bind_interaction: bool =True, # Whether the sub-plots for each variable should be connected for zooming/panning
        props:dict = None # additional properties (eg. size) for altair plot
       ) -> alt.Chart:
    
    df = self.tidy()
    return facet_variable(df, n_cols, bind_interaction, props)

In [ ]:
m_df.show()

alt.VConcatChart(...)

In [ ]:
a_gap(blk(799)).show()

alt.VConcatChart(...)

In [ ]:
idx = L(*blk(1).columns).argwhere(lambda x: x in ['TA','SW_IN'])

In [ ]:
mask = np.ones_like(blk(1), dtype=bool)

In [ ]:
mask

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [ ]:
gap = _make_random_gap(2, 10, 2)

In [ ]:
gap

array([False, False,  True,  True, False, False, False, False, False,
       False])

In [ ]:
np.argwhere(gap)

array([[2],
       [3]])

In [ ]:
mask[np.argwhere(gap), idx] = False

In [ ]:
mask

array([[ True,  True,  True],
       [ True,  True,  True],
       [False, False,  True],
       [False, False,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [ ]:
mask[gap]

array([[False, False,  True],
       [False, False,  True]])

### 3) To Tensor

this needs to handle both the init with a list of items and when the first item is a sequence of list of items

In [ ]:
class MaskedTensor(collections.abc.Sequence):
    def __init__(self,*args):
        if len(args)==2:
            self.data = args[0]
            self.mask = args[1]
        elif len(args)==1 and len(args[0])==2:
            self.data = args[0][0]
            self.mask = args[0][1]
        else:
            raise ValueError(f"Incorrect number of arguments. got {len(args)} args")

    def __iter__(self): return iter((self.data, self.mask,))
    __len__ = 2
    def __getitem__(self, key):
        if key == 0: return self.data
        elif key == 1: return self.mask
        else: raise IndexError("index bigger than 2")
    __repr__ = basic_repr('data, mask')

In [ ]:
#| export
class MaskedDf2Tensor(Transform):
    def setups(self, items):
        self.columns = list(items[0].data.columns)
    def encodes(self, df: MaskedDf) -> MaskedTensor:
        data = torch.tensor(df.data.to_numpy())
        mask = torch.tensor(df.mask.to_numpy())
        return MaskedTensor(data, mask)
        
    def decodes(self, x: MaskedTensor) -> MaskedDf:
        data = pd.DataFrame(x.data.detach().cpu().numpy(), columns = self.columns)
        mask = pd.DataFrame(x.mask.cpu().numpy(), columns = self.columns)
        return MaskedDf(data, mask)

In [ ]:
to_t = MaskedDf2Tensor()

In [ ]:
to_t.setup(tfms2)

In [ ]:
to_t(tfms2[0])

__main__.MaskedTensor(data=tensor([[1.4650e+01, 4.6819e+02, 6.4540e+00],
        [1.4220e+01, 2.2480e+02, 5.7990e+00],
        [1.4110e+01, 1.9528e+02, 6.5770e+00],
        [1.4230e+01, 2.4417e+02, 6.9310e+00],
        [1.4400e+01, 2.5392e+02, 7.2860e+00],
        [1.4090e+01, 1.7731e+02, 7.2510e+00],
        [1.3710e+01, 9.7070e+01, 6.6830e+00],
        [1.3080e+01, 3.9710e+01, 5.8510e+00],
        [1.2410e+01, 1.0650e+01, 5.2540e+00],
        [1.2270e+01, 3.2000e-01, 5.1640e+00]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
to_t.decode(to_t(tfms2[0]));

In [ ]:
tfms2[0]

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


In [ ]:
type(MaskedDf2Tensor())

__main__.MaskedDf2Tensor

In [ ]:
tfms3 = TfmdLists([800, 801, 802], [BlockDfTransform(hai, 10), AddGapTransform(['TA','SW_IN'], 2), MaskedDf2Tensor()])

In [ ]:
tfms3[0]

__main__.MaskedTensor(data=tensor([[1.4650e+01, 4.6819e+02, 6.4540e+00],
        [1.4220e+01, 2.2480e+02, 5.7990e+00],
        [1.4110e+01, 1.9528e+02, 6.5770e+00],
        [1.4230e+01, 2.4417e+02, 6.9310e+00],
        [1.4400e+01, 2.5392e+02, 7.2860e+00],
        [1.4090e+01, 1.7731e+02, 7.2510e+00],
        [1.3710e+01, 9.7070e+01, 6.6830e+00],
        [1.3080e+01, 3.9710e+01, 5.8510e+00],
        [1.2410e+01, 1.0650e+01, 5.2540e+00],
        [1.2270e+01, 3.2000e-01, 5.1640e+00]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
type(tfms3[0])

__main__.MaskedTensor

In [ ]:
tfms3.decode(tfms3[0])

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


### 4) Normalize

In [ ]:
#| export
from meteo_imp.utils import *
from fastai.torch_core import to_cpu

from torch import Tensor

In [ ]:
collections.namedtuple

<function collections.namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)>

In [ ]:
nt = collections.namedtuple("nt", "a")

In [ ]:
isinstance(nt(1), tuple)

True

In [ ]:
class NormalsParams(list):
    def __init__(self,*args):
        if len(args)==2:
            self.mean = args[0]
            self.std = args[1]
        elif isinstance(args[0], Generator):
            args = list(args[0])
            self.mean = args[0]
            self.std = args[1]
        elif len(args)==1 and len(args[0])==2:
            self.mean = tuple(args[0])[0]
            self.std = tuple(args[0])[1]                     
        else:
            raise ValueError(f"Incorrect number of arguments. got {len(args)} args")
    def __iter__(self): return iter((self.mean, self.std,))
    def __next__(self): return next(self.__iter__())
    def __len__(self): return 2
    def __getitem__(self, key):
        if key == 0: return self.mean
        elif key == 1: return self.std
        else: raise IndexError("index bigger than 2")
    __repr__ = basic_repr('mean, std')

In [ ]:
NormalsParams(0,1)

__main__.NormalsParams(mean=0, std=1)

In [ ]:
#| export
def get_stats(df, device='cpu'):
    return torch.tensor(df.mean(axis=0).to_numpy(), device=device), torch.tensor(df.std(axis=0).to_numpy(), device=device)

In [ ]:
#| export
class NormalizeMasked(Transform):
    "Normalize/denorm MaskedTensor column-wise "
    @property
    def name(self): return f"{super().name} -- {getattr(self,'__stored_args__',{})}"

    def __init__(self, mean=None, std=None): store_attr()

    def encodes(self, x:MaskedTensor)-> MaskedTensor:
        return MaskedTensor((x.data -self.mean) / self.std, x.mask)

    def decodes(self, x:MaskedTensor)->MaskedTensor:
        f = to_cpu if x[0].device.type=='cpu' else noop
        return MaskedTensor(x[0] * f(self.std) + f(self.mean), x[1])
    
    def decodes(self, x:NormalsParams):
        print("decoding")
        f = partial(to_device, device=(x[0].device))
        mean = x.mean * f(self.std) + f(self.mean)
        std = x.std * f(self.std)
        
        return NormalsParams(mean, std)

In [ ]:
norm = NormalizeMasked(*get_stats(hai))

In [ ]:
tfms3[0]

__main__.MaskedTensor(data=tensor([[1.4650e+01, 4.6819e+02, 6.4540e+00],
        [1.4220e+01, 2.2480e+02, 5.7990e+00],
        [1.4110e+01, 1.9528e+02, 6.5770e+00],
        [1.4230e+01, 2.4417e+02, 6.9310e+00],
        [1.4400e+01, 2.5392e+02, 7.2860e+00],
        [1.4090e+01, 1.7731e+02, 7.2510e+00],
        [1.3710e+01, 9.7070e+01, 6.6830e+00],
        [1.3080e+01, 3.9710e+01, 5.8510e+00],
        [1.2410e+01, 1.0650e+01, 5.2540e+00],
        [1.2270e+01, 3.2000e-01, 5.1640e+00]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
test_close(norm.decode(norm(tfms3[0]))[0], tfms3[0][0], eps=2e-5)

Test that NormalsParams decode actually works

In [ ]:
Npars = NormalsParams(torch.tensor(1), torch.tensor(.1))

In [ ]:
norm.decode(Npars)

decoding


__main__.NormalsParams(mean=tensor([ 16.2585, 324.9604,   7.7491]), std=tensor([ 0.7925, 20.4003,  0.4368]))

In [ ]:
tfms4 = TfmdLists([800,801,803], [BlockDfTransform(hai, 10), 
                           AddGapTransform(['TA','SW_IN'], 2),
                           MaskedDf2Tensor(),
                           NormalizeMasked(*get_stats(hai,device='cpu'), ) ])

In [ ]:
tfms4[0]

__main__.MaskedTensor(data=tensor([[ 0.7970,  1.7021,  0.7035],
        [ 0.7428,  0.5090,  0.5536],
        [ 0.7289,  0.3643,  0.7317],
        [ 0.7440,  0.6040,  0.8127],
        [ 0.7655,  0.6518,  0.8940],
        [ 0.7264,  0.2762,  0.8860],
        [ 0.6784, -0.1171,  0.7560],
        [ 0.5989, -0.3983,  0.5655],
        [ 0.5144, -0.5407,  0.4288],
        [ 0.4967, -0.5914,  0.4082]]), mask=tensor([[ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [False, False,  True],
        [False, False,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True]]))

In [ ]:
tfms4.decode(tfms4[0])

TA,SW_IN,VPD
14.6500,468.1900,6.4540
14.2200,224.8000,5.7990
14.1100,195.2800,6.5770
14.2300,244.1700,6.9310
14.4000,253.9200,7.2860
14.0900,177.3100,7.2510
13.7100,97.0700,6.6830
13.0800,39.7100,5.8510
12.4100,10.6500,5.2540
12.2700,0.3200,5.1640


is workinggggggggggggggggg 

### 5) To Tuple

Fastai likes to work with tuples ... for now convert to a tuple. Should add a decode step and maybe find a way to mimic a tuple in MaskedTensor

In [ ]:
def to_tuple(x):
    return tuple(x)

### Pipeline

In [ ]:
#| export
from fastai.data.transforms import *

In [ ]:
block_len = 10
block_ids = list(range(0, (len(hai) // block_len) - 1))[:10]
gap_len = 2

In [ ]:
#| export
def imp_pipeline(df,
                 block_len,
                 gap_len
                ):
    block_ids = list(range(0, (len(df) // block_len) - 1))
    return [BlockDfTransform(df, block_len),
            AddGapTransform(['TA','SW_IN'], gap_len),
            MaskedDf2Tensor,
            NormalizeMasked(*get_stats(df)),
            to_tuple
           ], block_ids

In [ ]:
pipeline, block_ids = imp_pipeline(hai, block_len, gap_len)

In [ ]:
pipeline

[BlockDfTransform:
 encodes: (int,object) -> encodes
 decodes: ,
 AddGapTransform:
 encodes: (DataFrame,object) -> encodes
 decodes: ,
 __main__.MaskedDf2Tensor,
 NormalizeMasked -- {'mean': tensor([  8.3339, 120.9578,   3.3807]), 'std': tensor([  7.9246, 204.0026,   4.3684])}:
 encodes: (MaskedTensor,object) -> encodes
 decodes: (MaskedTensor,object) -> decodes
 (NormalsParams,object) -> decodes,
 <function __main__.to_tuple(x)>]

In [ ]:
pp = Pipeline(pipeline)

In [ ]:
pp

Pipeline: BlockDfTransform -> AddGapTransform -> MaskedDf2Tensor -> NormalizeMasked -- {'mean': tensor([  8.3339, 120.9578,   3.3807]), 'std': tensor([  7.9246, 204.0026,   4.3684])} -> to_tuple

### Dataloader

random splitter for validation/training set

In [ ]:
reset_seed()

In [ ]:
splits = RandomSplitter()(block_ids)

Repeat twice the pipeline since is the same pipeline both for training data and for labels

In [ ]:
import collections

In [ ]:
def to_tuple(x):
    return tuple(x)

In [ ]:
isinstance(tfms4[0], Sequence)

True

In [ ]:
ds = Datasets(block_ids, [pipeline, pipeline], splits=splits)

In [ ]:
dls = ds.dataloaders(bs=1)

In [ ]:
dls.device

device(type='cuda', index=0)

In [ ]:
dls.one_batch()

((tensor([[[ 0.7869, -0.2606, -0.4500],
           [ 0.8576,  0.0989, -0.4088],
           [ 0.8702, -0.4311, -0.3669],
           [ 0.9825, -0.4901, -0.0116],
           [ 0.9043, -0.5859, -0.2657],
           [ 0.8286, -0.5929, -0.4557],
           [ 0.7920, -0.5929, -0.5367],
           [ 0.7794, -0.5929, -0.5686],
           [ 0.7718, -0.5929, -0.5656],
           [ 0.7617, -0.5929, -0.5853]]], device='cuda:0'),
  tensor([[[False, False,  True],
           [False, False,  True],
           [ True,  True,  True],
           [ True,  True,  True],
           [ True,  True,  True],
           [ True,  True,  True],
           [ True,  True,  True],
           [ True,  True,  True],
           [ True,  True,  True],
           [ True,  True,  True]]], device='cuda:0')),
 (tensor([[[ 0.7869, -0.2606, -0.4500],
           [ 0.8576,  0.0989, -0.4088],
           [ 0.8702, -0.4311, -0.3669],
           [ 0.9825, -0.4901, -0.0116],
           [ 0.9043, -0.5859, -0.2657],
           [ 0.8286

In [ ]:
@typedispatch
def show_batch(x: MaskedDf, y, samples, ctxs=None, max_n=6):
    return x

In [ ]:
# dls.show_batch()

In [ ]:
dls._types

{tuple: [{tuple: [torch.Tensor, torch.Tensor]},
  {tuple: [torch.Tensor, torch.Tensor]}]}

In [ ]:
Datasets

fastai.data.core.Datasets

In [ ]:
#| export
def make_dataloader(df, block_len, gap_len, bs=10):
    pipeline, block_ids = imp_pipeline(df, block_len, gap_len)
    
    splits = RandomSplitter()(block_ids)
    ds = Datasets(block_ids, [pipeline, pipeline], splits=splits)
    
    return ds.dataloaders(bs=bs)
    

In [ ]:
dls = make_dataloader(hai, 200, 10)

In [ ]:
dls.one_batch()[0][0].shape

torch.Size([10, 200, 3])

In [ ]:
dls = dls.cpu()

## Model

### Forward Function

in order to the a pytorch module we need a forward method to the kalman filter

In [ ]:
#| export
from meteo_imp.kalman.filter import *
from torch.distributions import MultivariateNormal

In [ ]:
#| export
@patch
def _predict_filter(self: KalmanFilter, data, mask):
    """Predict every obsevation using only the filter step"""
    # use the predicted state not the filtered state!
    obs, mask = self._parse_obs(data, mask)
    pred_state_mean, pred_state_cov, _, _ = self._filter_all(obs, mask)
    mean, cov = self._obs_from_state(ListMNormal(pred_state_mean.squeeze(-1), pred_state_cov))
    
    return ListNormal(mean, cov2std(cov))

In [ ]:
model = KalmanFilter.init_simple(n_dim = hai.shape[-1])

In [ ]:
model._predict_filter(*dls.one_batch()[0]);

In [ ]:
#| export
@patch
def forward(self: KalmanFilter, masked_data: MaskedTensor):
    data, mask = masked_data
    assert not data.isnan().any()
    use_smooth = self.use_smooth if hasattr(self, 'use_smooth') else True
    
    mean, std = (self.predict(obs=data, mask=mask, smooth=True) if use_smooth
                        else self._predict_filter(data, mask))
    return NormalsParams(mean, std) # to have fastai working this needs to be a tuple subclass

In [ ]:
input = dls.one_batch()[0]
target = dls.one_batch()[1]

In [ ]:
model.state_dict()

OrderedDict([('trans_matrix',
              tensor([[1., 0., 0.],
                      [0., 1., 0.],
                      [0., 0., 1.]])),
             ('trans_off', tensor([0., 0., 0.])),
             ('trans_cov_raw',
              tensor([[1., 0., 0.],
                      [0., 1., 0.],
                      [0., 0., 1.]])),
             ('obs_matrix',
              tensor([[1., 0., 0.],
                      [0., 1., 0.],
                      [0., 0., 1.]])),
             ('obs_off', tensor([0., 0., 0.])),
             ('obs_cov_raw',
              tensor([[1., 0., 0.],
                      [0., 1., 0.],
                      [0., 0., 1.]])),
             ('init_state_mean', tensor([0., 0., 0.])),
             ('init_state_cov_raw',
              tensor([[1., 0., 0.],
                      [0., 1., 0.],
                      [0., 0., 1.]]))])

In [ ]:
data = input[0][0]
data.shape

torch.Size([200, 3])

In [ ]:
mask = input[1][0]

In [ ]:
mask.shape

torch.Size([200, 3])

In [ ]:
data.device

device(type='cpu')

In [ ]:
torch.device

torch.device

In [ ]:
data.shape, mask.shape

(torch.Size([200, 3]), torch.Size([200, 3]))

In [ ]:
model.predict(data, mask);

In [ ]:
model.use_smooth = True

In [ ]:
pred = model(input)

In [ ]:
pred[0].shape

torch.Size([10, 200, 3])

In [ ]:
pred[1].shape

torch.Size([10, 200, 3])

In [ ]:
model.use_smooth = False

In [ ]:
pred_filt = model(input)

In [ ]:
pred_filt[1].shape

torch.Size([10, 200, 3])

In [ ]:
pred

__main__.NormalsParams(mean=tensor([[[-0.0823, -0.0877, -0.4107],
         [-0.0963,  0.0131, -0.5504],
         [-0.0887,  0.1950, -0.5828],
         ...,
         [ 0.4090,  2.7141,  0.3547],
         [ 0.4871,  3.0159,  0.5083],
         [ 0.5392,  3.0718,  0.6512]],

        [[-0.5066, -0.1224, -0.3928],
         [-0.6953, -0.0966, -0.5470],
         [-0.7637, -0.0231, -0.6088],
         ...,
         [-0.4738,  0.5079, -0.3268],
         [-0.4589,  0.5540, -0.3147],
         [-0.4455,  0.5257, -0.3078]],

        [[-0.1031, -0.0832, -0.0105],
         [-0.1309,  0.1104, -0.0090],
         [-0.0973,  0.5301,  0.0344],
         ...,
         [-0.5122, -0.2669, -0.5620],
         [-0.4926, -0.2394, -0.5387],
         [-0.4832, -0.2535, -0.5367]],

        ...,

        [[ 0.0852,  0.3574, -0.2978],
         [ 0.1487,  0.5134, -0.3731],
         [ 0.1835,  0.3665, -0.3654],
         ...,
         [ 1.1900,  1.7879,  1.2618],
         [ 1.1581,  1.0269,  1.1978],
         [ 1.1265,  0.

In [ ]:
type(pred), type(pred_filt)

(__main__.NormalsParams, __main__.NormalsParams)

In [ ]:
test_ne(pred, pred_filt)

### Loss Function

add support for complete loss (also outside gap) and for filter loss (don't run the smooher)

There are two ways to compute the loss, one is to do it for all predictions the other is for doing it for only the gap
- only_gap

Play around with flatting + diagonal

In [ ]:
a = torch.diag(torch.tensor([1,2,3]))
d = torch.stack([a, a*10])
m = torch.stack([a.diag(), a.diag()*10])
d

tensor([[[ 1,  0,  0],
         [ 0,  2,  0],
         [ 0,  0,  3]],

        [[10,  0,  0],
         [ 0, 20,  0],
         [ 0,  0, 30]]])

In [ ]:
m.flatten()

tensor([ 1,  2,  3, 10, 20, 30])

In [ ]:
d

tensor([[[ 1,  0,  0],
         [ 0,  2,  0],
         [ 0,  0,  3]],

        [[10,  0,  0],
         [ 0, 20,  0],
         [ 0,  0, 30]]])

In [ ]:
torch.diagonal(d, dim1=1, dim2=2).flatten()

tensor([ 1,  2,  3, 10, 20, 30])

In [ ]:
means, stds = pred
data, mask = target

In [ ]:
# make a big matrix with all variables and observations and compute ll
mask = mask.flatten() 
obs = data.flatten()[mask]
means = data.flatten()[mask]
stds = stds.flatten()[mask] # need to support batches

MultivariateNormal(means, torch.diag(stds)).log_prob(obs)

tensor(-5930.4453, grad_fn=<SubBackward0>)

In [ ]:
#| export
class KalmanLoss():
    def __init__(self,
                 only_gap:bool=True, # loss for all predictions or only gap
                 reduction:str='sum' # one of ['sum', 'mean', 'none']
                ):
        store_attr()
    
    def __call__(self, pred: NormalsParams, target: MaskedTensor):
        data, mask = target
        means, stds = pred        
        assert not stds.isnan().any()
        losses = torch.empty(data.shape[0], device=data.device, dtype=data.dtype)
        for i, (d, m, mean, std) in enumerate(zip(data, mask, means, stds)):
            losses[i] = self._loss_batch(d,m,mean, std)
        if self.reduction == 'none': return losses
        elif self.reduction == 'mean': return losses.mean()
        elif self.reduction == 'sum': return losses.sum()
    
    def _loss_batch(self, data, mask, mean, std):
        # make a big vector with all variables and observations and compute ll
        mask = mask.flatten() if self.only_gap else torch.fill(mask, True).flatten()
        obs = data.flatten()[mask]
        mean = data.flatten()[mask]
        std = std.flatten()[mask] 
        
        return MultivariateNormal(mean, torch.diag(std)).log_prob(obs)
        

In [ ]:
means, stds = pred

In [ ]:
stds.shape

torch.Size([10, 200, 3])

In [ ]:
means.shape

torch.Size([10, 200, 3])

In [ ]:
data.isnan().any()

tensor(False)

In [ ]:
mask.isnan().any()

tensor(False)

In [ ]:
means.isnan().any()

tensor(False)

In [ ]:
stds.isnan().sum()

tensor(0)

In [ ]:
stds

tensor([[[1.1756, 1.1756, 1.1756],
         [1.1990, 1.1990, 1.1990],
         [1.2024, 1.2024, 1.2024],
         ...,
         [1.2045, 1.2045, 1.2045],
         [1.2133, 1.2133, 1.2133],
         [1.2720, 1.2720, 1.2720]],

        [[1.1756, 1.1756, 1.1756],
         [1.1990, 1.1990, 1.1990],
         [1.2024, 1.2024, 1.2024],
         ...,
         [1.2045, 1.2045, 1.2045],
         [1.2133, 1.2133, 1.2133],
         [1.2720, 1.2720, 1.2720]],

        [[1.1756, 1.1756, 1.1756],
         [1.1990, 1.1990, 1.1990],
         [1.2024, 1.2024, 1.2024],
         ...,
         [1.2045, 1.2045, 1.2045],
         [1.2133, 1.2133, 1.2133],
         [1.2720, 1.2720, 1.2720]],

        ...,

        [[1.1756, 1.1756, 1.1756],
         [1.1990, 1.1990, 1.1990],
         [1.2024, 1.2024, 1.2024],
         ...,
         [1.2045, 1.2045, 1.2045],
         [1.2133, 1.2133, 1.2133],
         [1.2720, 1.2720, 1.2720]],

        [[1.1756, 1.1756, 1.1756],
         [1.1990, 1.1990, 1.1990],
         [1.

In [ ]:
is_posdef_eigv(torch.diag(stds.flatten()))

(tensor(True),
 tensor([1.1756, 1.1756, 1.1756,  ..., 2.0096, 2.0096, 2.0096],
        grad_fn=<LinalgEighBackward0>))

In [ ]:
KalmanLoss(only_gap=True)(pred, target)

tensor(-5930.4453, grad_fn=<SumBackward0>)

In [ ]:
KalmanLoss(only_gap=False)(pred, target)

tensor(-6132.7690, grad_fn=<SumBackward0>)

### Metrics

Wrapper around fastai metrics to support masked tensors and normal distributions

In [ ]:
#| export
def to_msk_metric(metric, name):
    def msk_metric(imp, targ):
        return metric(imp[0], targ[0]) # first element are the means
    msk_metric.__name__ = name
    return msk_metric

In [ ]:
from fastai.metrics import *

In [ ]:
msk_rmse = to_msk_metric(rmse, 'rmse')

In [ ]:
msk_rmse.__name__

'rmse'

In [ ]:
msk_rmse(pred, target)

TensorBase(1.5663)

In [ ]:
msk_r2 = to_msk_metric(R2Score(), 'r2')

In [ ]:
msk_r2(pred, target)

-1.5087013192978587

### Callback

save the model state 

In [ ]:
#| export
from fastai.callback.all import *

In [ ]:
#| export
class SaveParams(Callback):
    def __init__(self, param_name):
        super().__init__()
        self.params = []
        self.param_name = param_name
    def after_batch(self):
        param = getattr(self.model, self.param_name).detach()
        self.params.append(param)

In [ ]:
#| export
class SaveParams(Callback):
    def __init__(self, param_name):
        super().__init__()
        self.params = []
        self.param_name = param_name
    def after_batch(self):
        param = getattr(self.model, self.param_name).detach()
        self.params.append(param)

In [ ]:
debug_preds = []

In [ ]:
class DebugPredCallback(Callback):
    order = 0
    def after_validate(self):
        if hasattr(self, 'gather_preds'):
            debug_preds.append(self.gather_preds.preds)

### Learner

In [ ]:
#| export
from fastai.learner import * 

from fastai.tabular.all import *

from fastai.tabular.learner import *

from fastai.callback.progress import ShowGraphCallback

In [ ]:
obs_cov_history = SaveParams('obs_cov')

In [ ]:
all_data = CollectDataCallback()

In [ ]:
model = KalmanFilter.init_random(n_dim_obs = hai.shape[1], n_dim_state = hai.shape[1]).cuda()

In [ ]:
model.use_smooth = False

In [ ]:
# model._set_constraint('obs_cov', model.obs_cov, train=False)

In [ ]:
pipeline, block_ids = imp_pipeline(hai[:20000], block_len, gap_len)
    
splits = RandomSplitter()(block_ids)
ds = Datasets(block_ids, [pipeline, pipeline], splits=splits)

In [ ]:
dls = ds.dataloaders(bs=10, device='cuda')

In [ ]:
dls.one_batch()[0][0].device

device(type='cuda', index=0)

In [ ]:
input, target = dls.one_batch()

In [ ]:
pred = model(input)
KalmanLoss()(pred, target)

tensor(-351.0051, device='cuda:0', grad_fn=<SumBackward0>)

In [ ]:
learn = Learner(dls, model, loss_func=KalmanLoss(only_gap=False), cbs = [DebugPredCallback] , metrics = [msk_rmse, msk_r2])

In [ ]:
learn.fit(1, 1e-3)

epoch,train_loss,valid_loss,rmse,r2,time
0,-465.879425,-485.078766,3.601483,-12.849337,00:08


#### Float64

In [ ]:
model64 = KalmanFilter.init_random(hai.shape[1], hai.shape[1], dtype=torch.float64).cuda()

In [ ]:
dls64 = make_dataloader(hai64, 200, 10, bs=100)

In [ ]:
input = dls64.one_batch()[0]
target = dls64.one_batch()[1]

In [ ]:
data, mask = input

In [ ]:
data.device

device(type='cuda', index=0)

In [ ]:
data.dtype

torch.float64

In [ ]:
model64.predict(data);

In [ ]:
pred = model64(input)

In [ ]:
KalmanLoss()(pred, target)

tensor(-54159.9322, device='cuda:0', dtype=torch.float64,
       grad_fn=<SumBackward0>)

In [ ]:
#| export
class Float64Callback(Callback):
    order = Recorder.order + 10 # run after Recorder 
    def before_fit(self):
        self.recorder.smooth_loss.val = torch.tensor(0, dtype=torch.float64) # default is a float 32

In [ ]:
model64.use_smooth = False

In [ ]:
learn64 = Learner(dls64, model64, loss_func=KalmanLoss(), cbs = [Float64Callback] )

In [ ]:
learn64.fit(1, 1e-3)

epoch,train_loss,valid_loss,time
0,-70869.044487,-65195.777042,00:33


### Predictions

The transformation pipeline is not working properly (there is a problem in `decode_batch` as the `_types` are more nested than the predictions, which results in an error) + the pipeline is anyway not reproducible + the test dataloaders seems that they are actually not deterministic .....
soo reimplement everything almost from scratch

see https://github.com/mone27/meteo_imp/blob/0335003405ec9bd3e3bd2641bc6d7924f34a0788/lib_nbs/kalman/10_fastai.ipynb for all details

In [ ]:
#| export
class NormalsDf:
    """DataFrames of Normal parameters (mean and std)"""
    def __init__(self, mean, std): store_attr()
    def tidy(self, prefix=""):
        """Tidy version"""
        mean = self.mean.reset_index().melt("time", value_name=prefix + "mean")
        std = self.std.reset_index().melt("time", value_name=prefix + "std")
        return pd.merge(mean, std, on=["time", "variable"])
    __repr__ = basic_repr("mean, std")

In [ ]:
pipe0, pipe1 = tfms4.fs[0,1], tfms4.fs[2,3] 

In [ ]:
pipe0, pipe1

((#2) [BlockDfTransform:
encodes: (int,object) -> encodes
decodes: ,AddGapTransform:
encodes: (DataFrame,object) -> encodes
decodes: ],
 (#2) [MaskedDf2Tensor:
encodes: (MaskedDf,object) -> encodes
decodes: (MaskedTensor,object) -> decodes
,NormalizeMasked -- {'mean': tensor([  8.3339, 120.9578,   3.3807]), 'std': tensor([  7.9246, 204.0026,   4.3684])}:
encodes: (MaskedTensor,object) -> encodes
decodes: (MaskedTensor,object) -> decodes
(NormalsParams,object) -> decodes
])

In [ ]:
#| export
def preds2df(preds, targs):
    """Final step to decode preds by getting a dataframe"""
    # preds this is a tuple (data, mask)
    out = []
    for pred, targ in zip(preds, targs):
        # convert to dataframe using structure for
        mean = pd.DataFrame(pred[0].squeeze(0).detach().cpu().numpy(), columns = targ.data.columns, index=targ.data.index)
        std = pd.DataFrame(pred[1].squeeze(0).detach().cpu().numpy(), columns = targ.data.columns, index=targ.data.index)
        out.append(NormalsDf(mean, std))
    return out

In [ ]:
#| export
def predict_items(items, learn, pipe0, pipe1):
    pipe0, pipe1 = Pipeline(pipe0), Pipeline(pipe1)
    preds, targs, losses = [], [], []
    for item in items:
        targ = pipe0(item)
        data, mask = pipe1(targ)
        input = MaskedTensor(data.cuda().unsqueeze(0), mask.cuda().unsqueeze(0))
        pred = model(input)
        loss = learn.loss_func(pred, input)
        # denormalize
        pred = pipe1.decode(pred)
        preds.append(pred), targs.append(targ), losses.append(loss)
        
    return preds2df(preds, targs), targs, losses
        

In [ ]:
preds, targs, losses = predict_items([0,1,3], learn, pipe0, pipe1)

decoding
decoding
decoding


this is the same data!!

In [ ]:
predict_items([0], learn, pipe0, pipe1)[1][0].data == predict_items([0], learn, pipe0, pipe1)[1][0].data

decoding
decoding


,TA,SW_IN,VPD
time,,,
2000-01-01 00:30:00,True,True,True
2000-01-01 01:00:00,True,True,True
2000-01-01 01:30:00,True,True,True
2000-01-01 02:00:00,True,True,True
2000-01-01 02:30:00,True,True,True
2000-01-01 03:00:00,True,True,True
2000-01-01 03:30:00,True,True,True
2000-01-01 04:00:00,True,True,True
2000-01-01 04:30:00,True,True,True


#### plot results

In [ ]:
#| export
def plot_result(pred, targ, loss, **kwargs):
    df = pd.merge(targ.tidy(), pred.tidy(), on=["time", "variable"])
    return facet_variable(df, error=True, **kwargs).properties(title=f"loss: {loss.item():.6f}")

In [ ]:
plot_result(preds[0], targs[0], torch.tensor(1))

alt.VConcatChart(...)

In [ ]:
#| export
def plot_results(preds, targs, losses):
    plots = [plot_result(targ, pred, loss, n_cols=1) for targ, pred, loss in zip(preds, targs, losses)]
    return alt.hconcat(*plots)

In [ ]:
plot_results(preds, targs, losses)

alt.HConcatChart(...)

#### Show Results

In [ ]:
random.choices(learn.dls.items, k=3)

[519, 1120, 534]

In [ ]:
#| export
def show_results(learn, n=3):
    items = random.choices(learn.dls.items, k=3)
    pipe0, pipe1 = learn.dls.fs[0][0,1], learn.dls.fs[0][2,3]
    return plot_results(*predict_items(items, learn, pipe0, pipe1))
    

In [ ]:
show_results(learn)

decoding
decoding
decoding


alt.HConcatChart(...)

In [ ]:
display_as_row(learn.model.get_info())

## Export 

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()